<a href="https://colab.research.google.com/github/jainary4/DS3_Datathon/blob/main/FNN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
# here we are going to implement our own custom neural network
# we will implement a 3 layer network that takes in a image of size 28x28 which we first flattern to 1-D
# it is sent to the first layer from the input which maps the pixels to 150 neurons and then
# it is sent to the second layer which maps the 150 neurons to 75 neurons and then to the last layer which maps it to 10 neurons
# the data set is of digit recongnition
# we are using pytorch for our implementation
class MultiLayerNN(nn.Module):
  def __init__(self, input, first, second, output):
    super(MultiLayerNN, self).__init__()
    self.f1= nn.Linear(input,first)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()
    self.f2= nn.Linear(first,second)
    self.f3= nn.Linear(second,output)

  def forward(self,img):
    # defines how the data flows across the layers and the activation functions across the layers
    flattern= img.view(-1,28*28).requires_grad_()
    x= self.f1(flattern)
    x= self.relu(x)
    x= self.f2(x)
    x= self.relu(x)
    x= self.f3(x) # not applying any activation in the last layer as we dont want an output of either 1 or 0 in the digit logit
    return x

model = MultiLayerNN(28*28,150,75,10)

# our neural network is set now and we have to load the data set and divide it into training and test and divide it into iterations and epochs
# we want to prevent overfitting and memory/cpu overload so we are split the data into batches and run 3 epochs of the entire dataset
# we also randomly seperate the data as train dataset and test dataset
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())
# the MNIST dataset has 60000 images so we need to divide it into batches and epoches to fit the data properly
# we will define each batch to be 100 images
batches=100
epoches= 6
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batches,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batches,
                                          shuffle=False)
# we also define the the learning rate and the gradient descent fucnction
learn_rate= 0.07
gradient= torch.optim.SGD(model.parameters(), lr=learn_rate)
loss_function = nn.CrossEntropyLoss()

# here is where we are going to train the model
count =0
for e in range(epoches):
  # for each epoch we need to define a batch of 100 image and there will be 600 such batches
  for i, (images, labels) in enumerate(train_loader):
    gradient.zero_grad()
    run = model(images)
    # we calculate te loss
    loss_value= loss_function(run,labels) # labels represent the true y value and run represents the predicted y value
    loss_value.backward()
    gradient.step() # to update parameters
    count=count+1
  if count%600==0:
    # now we are testing the data
    total=0
    correct_result=0
    for images, labels in test_loader:
      # follow the same steps as in the time of training the data but now we record the values and test them against the true original values
      output= model(images)
      predicted = torch.max(output.data, 1) # this function takes the max logit value in the last layer as the value predicted by the neural network
      total += labels.size(0)
      #correct_result += (predicted == labels).sum()
      for i in range(len(labels)):
       if(predicted.indices[i] == labels[i]):
         correct_result=  correct_result+1


    accuracy = 100 * (correct_result /total)
    print(total)
    print(f"iteration_number={count} accuracy={accuracy}%")
    print("Predicted values:", predicted.indices)
    print("True values:", labels)








10000
iteration_number=600 accuracy=90.71000000000001%
Predicted values: tensor([8, 9, 0, 1, 2, 7, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 2, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 0, 2, 3, 8, 9, 9, 1, 4,
        0, 6, 1, 0, 0, 0, 2, 1, 1, 7, 7, 7, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 3, 2,
        4, 9, 4, 2, 6, 4, 1, 7, 3, 6, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6])
True values: tensor([8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 3, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 5, 3, 3, 3, 9, 1, 4,
        0, 6, 1, 0, 0, 6, 2, 1, 1, 7, 7, 8, 4, 6, 0, 7, 0, 3, 6, 8, 7, 1, 5, 2,
        4, 9, 4, 3, 6, 4, 1, 7, 2, 6, 5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2,
        3, 4, 5, 6])
10000
iteration_number=1200 accuracy=93.16%
Predicted values: tensor([8, 4, 0, 1, 2, 7, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 7, 8, 9, 7, 8, 6, 4, 1,
        9, 2, 8, 4, 4, 7, 0, 1, 9, 2, 8, 7, 8, 2, 6, 0, 6, 6, 3, 8, 9, 9, 1, 4,
        0,